### Génération d’embeddings

##### 1. Le choix de modèles génération d’embeddings

Parmi les meilleurs modèles de génération d’embeddings, on trouve :

| Modèle              | Qualité         | Multilingue       | Vitesse |
| ------------------- | --------------- | ----------------- | ------- |
| BGE-large-m3        | Excellent       | Oui               | Lent |
| E5-large-v2         | Excellent       | EN only           | Moyen |
| Multilingual-E5     | Bien            | Oui               | Moyen |
| GTE-large           | Bien            | EN only           | Rapide |
| MiniLM-L12-v2       | qualité faible  | Oui               | Très rapide |


On va utiliser `E5-large-v2` pour trouver un équilibre entre la performance et la vitesse d’entraînement.

##### 2. Prétraitement des textes pour le modéle `E5-large-v2`

Le modèle `E5-large-v2` a été entraîné sur du texte naturel (phrases réelles, majuscules, ponctuation, etc.), donc il faut garder le texte aussi proche que possible de sa forme originale,
tout en nettoyant les erreurs mineures ou les incohérences.

il est recommander d'éviter de:
- Supprimer les stopwords: Le modèle comprend leur rôle sémantique.
- Lemmatisation ou stemming: Cela modifie la structure linguistique et le sens.
- Supprimer la ponctuation: La ponctuation aide à comprendre le contexte.
- Tout mettre en minuscules (Sauf si le texte contient des majuscules aléatoires ou tout en majuscules.): Peut faire perdre des distinctions importantes (Apple ≠ apple, US ≠ us).
- Supprimer les chiffres: Les nombres ont souvent une signification utile (dates, montants…).

Il est Recommander (et parfois obligatoire) de:
- Supprimer les espaces inutiles: Nettoyer les débuts/fin de lignes et les espaces multiples.
- Gérer les valeurs manquantes: Remplacer les `NaN` ou textes vides par une chaîne vide.
- Ajouter le préfixe (Obligatoire): **"passage: "** pour les documents, **"query: "** pour les requêtes.
- Tronquer les textes très longs: Si un texte dépasse la limite (512 tokens environ), on peut le couper.

In [8]:
import pandas as pd

data_train = pd.read_csv('../data/raw/train.csv')
data_test = pd.read_csv('../data/raw/test.csv')


data_train["text"] = data_train["text"].fillna("").str.strip()
data_test["text"] = data_test["text"].fillna("").str.strip()


data_train["text"] = data_train["text"].str.replace(r"\s+", " ", regex=True)
data_test["text"] = data_test["text"].str.replace(r"\s+", " ", regex=True)


data_train["text_prefixed"] = "passage: " + data_train["text"]
data_test["text_prefixed"] = "passage: " + data_test["text"]

- Sauvegarder les modifications

In [10]:
data_train.to_csv('../data/processed/preprocessed_train_text.csv', index=False)
data_test.to_csv('../data/processed/preprocessed_test_text.csv', index=False)

##### 3. Charger le modèle `e5-large-v2` avec **Sentence Transformers**

In [1]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("intfloat/e5-large-v2")

c:\Users\anass\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'runtime_version' from 'google.protobuf' (c:\Users\anass\AppData\Local\Programs\Python\Python311\Lib\site-packages\google\protobuf\__init__.py)

##### 4. Générer les embeddings pour toutes les données.

- Train

In [ ]:
data_train["embedding"] = data_train["text_prefixed"].apply(
    lambda x: model.encode(x, normalize_embeddings=True)
)

- Test

In [ ]:
data_test["embedding"] = data_test["text_prefixed"].apply(
    lambda x: model.encode(x, normalize_embeddings=True)
)

Malheureusement, la génération des embeddings prend beaucoup de temps (plus de 400 minutes pour les données d'entraînement sans se terminer). Nous allons donc utiliser un autre modèle qui est plus rapide.

##### 5. Générer les embeddings avec le modèle `paraphrase-multilingual-MiniLM-L12-v2`

Prétraiter le texte:

- Supprimer les espaces inutiles: Nettoyer les débuts/fin de lignes et les espaces multiples.
- Gérer les valeurs manquantes: Remplacer les `NaN` ou textes vides par une chaîne vide.

In [3]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

- Train

In [ ]:
data_train["text"] = data_train["text"].fillna("").str.strip()
data_train["text"] = data_train["text"].str.replace(r"\s+", " ", regex=True)

texts = data_train["text"].tolist()

train_embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

Batches: 100%|██████████| 3750/3750 [1:19:32<00:00,  1.27s/it]


In [ ]:
import numpy as np
import os 

os.makedirs('../data/embeddings', exist_ok=True)

np.save("../data/embeddings/train_embeddings_minilm.npy", train_embeddings)

- Test

In [ ]:
data_test["text"] = data_test["text"].fillna("").str.strip()
data_test["text"] = data_test["text"].str.replace(r"\s+", " ", regex=True)

texts = data_test["text"].tolist()

test_embeddings = model.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

Batches: 100%|██████████| 238/238 [02:04<00:00,  1.92it/s]


In [ ]:
import numpy as np
import os 

os.makedirs('../data/embeddings', exist_ok=True)

np.save("../data/embeddings/test_embeddings_minilm.npy", test_embeddings)

- Sauvegardez le modéle

In [4]:
import joblib as jb

jb.dump(model, '../models/minilm_embedding_model.pkl')

['../models/minilm_embedding_model.pkl']

##### 6. Vérification des embeddings générés

In [56]:
print(train_embeddings.shape, '\n')
print(train_embeddings)

(120000, 384) 

[[ 0.07367805 -0.02631132  0.03900679 ... -0.07857122  0.05455048
   0.0741483 ]
 [-0.00685681 -0.06906829 -0.04574033 ... -0.06036082  0.04355336
   0.04123305]
 [-0.01750736 -0.03781662  0.024446   ... -0.11092535 -0.02253907
   0.04098494]
 ...
 [-0.02288848  0.01629857 -0.03647588 ...  0.03625313  0.02009865
   0.04081287]
 [-0.03029879 -0.02274068 -0.00463905 ...  0.01157964 -0.01234239
   0.02648926]
 [-0.03888905 -0.05788028 -0.02970092 ...  0.0345937   0.02085045
  -0.06742197]]


In [57]:
print(test_embeddings.shape, '\n')
print(test_embeddings)

(7600, 384) 

[[-0.02000974  0.14546126 -0.02131354 ... -0.0077236   0.01787492
   0.03831412]
 [-0.01230167  0.06320839 -0.03357151 ... -0.03042849 -0.1500939
   0.0277649 ]
 [-0.10158303  0.0071192  -0.04032289 ...  0.030038    0.07575675
   0.06632863]
 ...
 [ 0.04286214  0.02248745  0.00892183 ...  0.02532074  0.09171098
  -0.02843746]
 [-0.01266828 -0.00183072  0.03382643 ... -0.09408851  0.04791066
   0.06749152]
 [ 0.02815873 -0.03605843  0.03330523 ... -0.10941093  0.00155406
  -0.04573366]]
